Week 4:
- Linearization of nonlinear functions (absolute values, either-or, products)
- Lagrange multipliers and the Karush-Kuhn-Tucker (KKT) conditions.
- Utility maximization, ridge regression, and support vector machines.

- If both the objective function and all the constraints are linear functions of the decision variables, the problem is a linear program (LP).
- Although it seems like this assumption is somewhat restrictive, there are a surprising variety of problems that can be solved.
  - Absolute value, minimax or maximin, minimizing the sum of deviations, floor/ceiling constraints.
- Key Insight: Problems must be reformulated!

# practice problem 1:
Church-Key Brewing Company® has two warehouses from which it distributes beer to seven bars. At the start of each week, the bars send their orders (in cases) to the brewery. They brewery then satisfies the orders by assigning cases from the two warehouses to each of the bars. The brewery would like to formulate an optimization problem to find an assignment that minimizes fuel costs.

For example, this week the brewery has 7000 cases at warehouse A and 8000 cases at warehouse B. The bars require 1000, 1800, 3600, 400, 1400, 2500, and 2000 cases, respectively. Which warehouse should supply each bar given that the number of cases supplied by one warehouse must be within 1200 cases of the other?

The total cost per case (cij):
| Bar/Warehouse | A    | B    |
|---------------|------|------|
| 1             | $2.00| $3.00|
| 2             | $4.00| $1.00|
| 3             | $5.00| $3.00|
| 4             | $2.00| $2.00|
| 5             | $1.00| $3.00|
| 6             | $2.50| $1.75|
| 7             | $1.90| $1.60|



There are four types of constraints:
1. Demand constraints
2. Supply constraints
3. Absolute value constraint
4. Non-negativity and integer constraints

In [1]:
from gurobipy import GRB
import gurobipy as gb


# Maximum difference between inventory positions of the warehouses 
max_diff = 1200

# The supply nodes (i.e., the warehouses)
W = 2

# Creates a list that has the number of units of supply for each supply node
supply = [7000, 8000]

# The demand nodes
B = 7

# Creates a list for the number of units of demand for each demand node
demand = [1000, 1800, 3600, 400, 1400, 2500, 2000]

# Creates a list of lists associated with the costs of each transportation path.
# From warehouse i = {A,B} to bar j = {1,2,3,4,5,6,7}. 
costs = [
         #Bars: 1 2 3 4 5 6 7
         [2.00,4.00,5.00,2.00,1.00,2.50,1.90],#A   Warehouses
         [3.00,1.00,3.00,2.00,3.00,1.75,1.60] #B
        ]

# Instantiate our optimization problem in
model = gb.Model("Linearize Absolute Value Constraint")

#Construct decision variables for each class of decision variables
x = model.addVars(W, B, lb = 0, vtype=GRB.INTEGER, name="Transportation")

# Add the objective function to the optimization problem 
model.setObjective(gb.quicksum(x[w,b]*costs[w][b] for w in range(W) for b in range(B)), GRB.MINIMIZE)

# The demand minimum constraints are added to the milp variable for each demand node (bar)
model.addConstrs(gb.quicksum(x[w,b] for w in range(W)) == demand[b] for b in range(B))

# The supply maximum constraints are added to the milp variable for each supply node (warehouse)
model.addConstrs(gb.quicksum(x[w,b] for b in range(B)) <= supply[w] for w in range(W))
                   
# The absolute value of the difference in the inventory supplied to the bars
# model.addConstr(abs(gb.quicksum(x[0,b] for b in range(B)) - gb.quicksum(x[1,b] for b in range(B)))  <= max_diff)
model.addConstr(gb.quicksum(x[0,b] for b in range(B)) - gb.quicksum(x[1,b] for b in range(B))  <= max_diff)
model.addConstr(gb.quicksum(x[1,b] for b in range(B)) - gb.quicksum(x[0,b] for b in range(B))  <= max_diff)

# Optimally solve the problem
model.optimize()

# Each of the variables is printed with it's resolved optimum value
total_supply = [0,0]
for v in model.getVars():    
    if ("[0," in v.varName):
        total_supply[0] += v.x
    else: 
        total_supply[1] += v.x

# The optimized objective function value is printed to the screen    
print("Total Cost of Transportation = ", model.objVal)
print("Supply from Warehouse A = ", total_supply[0])
print("Supply from Warehouse B = ", total_supply[1])
print("Supply Difference = ", abs(total_supply[0]-total_supply[1]))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-20
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11 rows, 14 columns and 56 nonzeros
Model fingerprint: 0x9f5170e3
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Found heuristic solution: objective 37975.000000
Presolve removed 9 rows and 7 columns
Presolve time: 0.00s
Presolved: 2 rows, 7 columns, 14 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective 34218.000000

Root relaxation: objective 2.568750e+04, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N

### Practice question 2
A company is involved in the production of two items (X and Y). The resources needed to produce X and Y are twofold, namely machine time for automatic processing and craftsman time for hand finishing. The table below gives the number of minutes required for each item:

|            | Machine time | Craftsman time |
|------------|--------------|----------------|
| Item X     | 13           | 20             |
| Item Y     | 19           | 29             |

The company has 40 hours of machine time available in the next working week but only 35 hours of craftsman time. Machine time is costed at £10 per hour worked and craftsman time is costed at £2 per hour worked. Both machine and craftsman idle times incur no costs. The revenue received for each item produced (all production is sold) is £20 for X and £30 for Y. The company has a specific contract to produce 10 items of X per week for a particular customer.

Formulate the problem of deciding how much to produce per week as a linear program.


In [2]:
model = gb.Model("Production Optimization")

# Define decision variables
x = model.addVar(vtype=gb.GRB.INTEGER, name="x", lb=10)  # Production quantity of item X
y = model.addVar(vtype=gb.GRB.INTEGER, name="y",lb=0)        # Production quantity of item Y

# Define constraints
machine_time = (13 * x + 19 * y)/60 # hours
craftsman_time = (20 * x + 29 * y)/60 # hours

model.addConstr(machine_time <= 40 , 'Machine time')    # Convert hours to minutes
model.addConstr(craftsman_time <= 35 , 'Craftsman time') # Convert hours to minutes

# Set objective function
model.setObjective(20 * x + 30 * y - (machine_time * 10 + craftsman_time * 2), gb.GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Print the optimal solution
print("Optimal solution:")
print("Production quantity of item X:", x.x)
print("Production quantity of item Y:", y.x)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9a5781f4
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e-01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [4e+01, 4e+01]
Found heuristic solution: objective 1802.5000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 1.865875e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1865.875

# practice question 3
Answer the questions related to the model below:

Maximize: \(3x_1 + 2x_2\)
Subject to:
\[
\begin{align*}
2x_1 + 2x_2 & \leq 5 \\
2x_1 + x_2 & \leq 4 \\
x_1 + 2x_2 & \leq 4 \\
x_1, x_2 & \geq 0
\end{align*}
\]

a. Use the simplex algorithm to find the optimal solution to the model.

b. Find the dual of the model.

In [3]:
import gurobipy as gb

# Define primal model
m_primal = gb.Model("Primal Problem")

# Define decision variables
x = m_primal.addVars(2, vtype=gb.GRB.INTEGER, name="x", lb=0)

# Set objective function
m_primal.setObjective(3 * x[0] + 2 * x[1], gb.GRB.MAXIMIZE)

# Add constraints
m_primal.addConstr(2 * x[0] + 2 * x[1] <= 5, "c1")
m_primal.addConstr(2 * x[0] + x[1] <= 4, "c2")
m_primal.addConstr(x[0] + 2 * x[1] <= 4, "c3")

# Optimize the primal model
m_primal.optimize()

# Print the optimal solution
print("Optimal solution for primal problem:")
print("x1:", x[0].x)
print("x2:", x[1].x)
print("Optimal objective value:", m_primal.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x19abd45a
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%
Optimal solution for primal problem:
x1: 2.0
x2:

In [4]:
# Define dual model
m_dual = gb.Model("Dual Problem")

# Define decision variables
y = m_dual.addVars(3, vtype=gb.GRB.INTEGER, name="y", lb=0)

# Set objective function
m_dual.setObjective(5 * y[0] + 4 * y[1] + 4 * y[2], gb.GRB.MINIMIZE)

# Add constraints
m_dual.addConstr(2 * y[0] + 2 * y[1] + y[2] >= 3, "x_dual")
m_dual.addConstr(2 * y[0] + y[1] + 2 * y[2] >= 2, "y_dual")

# Optimize the dual model
m_dual.optimize()

# Print the optimal solution
print("\nOptimal solution for dual problem:")
print("y1:", y[0].x)
print("y2:", y[1].x)
print("y3:", y[2].x)
print("Optimal objective value:", m_dual.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x9e1ab566
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [4e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+00]
Found heuristic solution: objective 12.0000000
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 6.500000e+00, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.50000    0    1   12.00000    6.50000  45.8

Maximize

𝒁=7𝑇+5𝐶

Subject to the constraints:

3𝑇+4𝐶≤2400

2𝑇+1𝐶≤1000

𝐶≤450

T≥100

𝑇,𝐶≥0


In [5]:
import gurobipy as gb

# Define primal model
m_primal = gb.Model("Primal Problem")

# Define decision variables
t = m_primal.addVar(vtype=gb.GRB.CONTINUOUS, name="t", lb=0)
c = m_primal.addVar( vtype=gb.GRB.CONTINUOUS, name="c", lb=0)

# Set objective function
m_primal.setObjective(7 * t + 5 * c, gb.GRB.MAXIMIZE)

# Add constraints
m_primal.addConstr(3 * t + 2 * c <= 2400, "c1")
m_primal.addConstr(2 * t + c <=1000 , "c2")
m_primal.addConstr(c <=450, "c3")
m_primal.addConstr(t >=100, "c3")

# Optimize the primal model
m_primal.optimize()

# Print the optimal solution
print("Optimal solution for primal problem:")
print("t:", t.x)
print("c:", c.x)
print("Optimal objective value:", m_primal.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x33d80bf3
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.9500000e+03   1.062500e+02   0.000000e+00      0s
       2    4.1750000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.175000000e+03
Optimal solution for primal problem:
t: 275.0
c: 450.0
Optimal objective value: 4175.0


In [6]:
import gurobipy as gb
import pandas as pd

# Define dual model
m_dual = gb.Model("Dual Problem")

# Define decision variables for the dual problem
y1 = m_dual.addVar(vtype=gb.GRB.CONTINUOUS, name="y1", lb=0)  # Non-negative
y2 = m_dual.addVar(vtype=gb.GRB.CONTINUOUS, name="y2", lb=0)  # Non-negative
y3 = m_dual.addVar(vtype=gb.GRB.CONTINUOUS, name="y3", lb=0)  # Non-negative
y4 = m_dual.addVar(vtype=gb.GRB.CONTINUOUS, name="y4", ub=0)  # Non-positive

# Set objective function for the dual problem
m_dual.setObjective(2400 * y1 + 1000 * y2 + 450 * y3 + 100 * y4, gb.GRB.MINIMIZE)

# Add constraints for the dual problem
m_dual.addConstr(3 * y1 + 2 * y2 - y4 >= 7, "dual_c1")
m_dual.addConstr(2 * y1 + y2 + y3 >= 5, "dual_c2")

# Optimize the dual model
m_dual.optimize()

# Print the optimal solution for the dual problem
if m_dual.status == gb.GRB.OPTIMAL:
    print("Optimal solution for dual problem:")
    for v in m_dual.getVars():
        print(f"{v.varName}: {v.x}")
    print("Optimal objective value:", m_dual.objVal)
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22635.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 4 columns and 6 nonzeros
Model fingerprint: 0x9fb01e1e
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 7e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.250000e+00   0.000000e+00      0s
       2    4.1750000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.175000000e+03
Optimal solution for dual problem:
y1: 0.0
y2: 3.5
y3: 1.5
y4: 0.0
Optimal objective value: 4175.0


In [7]:
# Extract the shadow prices and slacks for all constraints
shadow_prices = m_primal.getAttr('Pi', m_primal.getConstrs())
slacks = m_primal.getAttr('Slack', m_primal.getConstrs())
primal_table_data = []
# Update the table data for constraints
for i, c in enumerate(m_primal.getConstrs()):
    constr_info = {
        "Constraint Name": c.ConstrName,
        "Shadow Price": shadow_prices[i],
        "Slack": slacks[i],
        "RHS Value": c.RHS
    }
    primal_table_data.append(constr_info)

# Convert to DataFrame
pd.DataFrame(primal_table_data)

,Constraint Name,Shadow Price,Slack,RHS Value
0,c1,0.0,675.0,2400.0
1,c2,3.5,0.0,1000.0
2,c3,1.5,0.0,450.0
3,c3,0.0,-175.0,100.0


In [8]:
# Extract the shadow prices and slacks for all constraints
shadow_prices = m_dual.getAttr('Pi', m_dual.getConstrs())
slacks = m_dual.getAttr('Slack', m_dual.getConstrs())
dual_table_data = []
# Update the table data for constraints
for i, c in enumerate(m_dual.getConstrs()):
    constr_info = {
        "Constraint Name": c.ConstrName,
        "Shadow Price": shadow_prices[i],
        "Slack": slacks[i],
        "RHS Value": c.RHS
    }
    dual_table_data.append(constr_info)

# Convert to DataFrame
pd.DataFrame(dual_table_data)

,Constraint Name,Shadow Price,Slack,RHS Value
0,dual_c1,275.0,0.0,7.0
1,dual_c2,450.0,0.0,5.0


In [9]:
# zero sum game rule
# KKT conditions
# Slack, Shadow pricing, 